# Developing with OpenAI: AIM Edition

## Exploring LLM Prompting Strategies for Economic Reasoning  
### *Inflation & Interest Rate Case Study*

This notebook investigates how different prompting strategies (zero-shot, few-shot, reasoning vs non-reasoning models) affect the ability of large language models (LLMs) to reason about inflation, interest rates, and overall market dynamics.  

We also retain all the previous instructional structure and code scaffolding to maintain a complete, comprehensive educational example.

## 1. Getting Started

The first thing we'll do is load the [OpenAI Python Library](https://github.com/openai/openai-python/tree/main)!

In [ ]:
# Used for Google Colab
#!pip install openai -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip



## Discussion and Problem Framing

We aim to answer:  
> *"What is the best prompting approach and model type to understand how the market is performing today?"*  

### Types of LLM Tasks Involved

| Type | Description | Example Output |
|------|--------------|----------------|
| **Retrieval** | Factual recall | “Inflation in 2025 is around 3.1% in the U.S.” |
| **Reasoning** | Logical chain between variables | “Higher inflation led the Fed to raise rates → borrowing costs rose → slower GDP.” |
| **Generation** | Narrative creation / summary | “The market shows cooling signals despite moderate inflation…” |

Each prompt and model will be evaluated on reasoning depth, factual correctness, and structure quality.


### Used models in this repo

| Rank | Model Name | Primary Purpose | OpenAI's Official Claim |
|------|------------|-----------------|------------------------|
| 1 | **GPT-5** | Advanced reasoning for complex economic analysis | Uses a dynamic router that chooses between quick responses and deeper 'thinking' when needed; performs at PhD-level across domains |
| 2 | **GPT-4.1** | Enhanced coding and long-context comprehension | Offers significant advancements in coding capabilities, long context comprehension (up to 1M tokens), and instruction following |
| 3 | **GPT-4-turbo** | General-purpose non-reasoning model for structured responses | Improved version of GPT-4 with enhanced performance, lower latency, and updated knowledge cutoff |
| 4 | **GPT-4o-mini** | Fast, efficient model for quick responses | Cost-efficient AI model designed to make advanced AI technology more affordable and accessible |


## 2. Setting Environment Variables

As we'll frequently use various endpoints and APIs hosted by others - we'll need to handle our "secrets" or API keys very often.

We'll use the following pattern throughout this bootcamp - but you can use whichever method you're most familiar with.

In [ ]:
# For Google Colab
# import os
# import getpass

# os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key")

In [ ]:
# For local development
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


## 3. Using the OpenAI Python Library

Let's jump right into it!

> NOTE: You can, and should, reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/authentication?lang=python) whenever you get stuck, have questions, or want to dive deeper.

### Creating a Client

The core feature of the OpenAI Python Library is the `OpenAI()` client. It's how we're going to interact with OpenAI's models, and under the hood of a lot what we'll touch on throughout this course.

> NOTE: We could manually provide our API key here, but we're going to instead rely on the fact that we put our API key into the `OPENAI_API_KEY` environment variable!

In [150]:
from openai import OpenAI

client = OpenAI()

### Using the Client

Now that we have our client - we're going to use the `.chat.completions.create` method to interact with the model.

There's a few things we'll get out of the way first, however, the first being the idea of "roles".

First it's important to understand the object that we're going to use to interact with the endpoint. It expects us to send an array of objects of the following format:

```python
{"role" : "ROLE", "content" : "YOUR CONTENT HERE", "name" : "THIS IS OPTIONAL"}
```

Second, there are three "roles" available to use to populate the `"role"` key:

- `system`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://help.openai.com/en/articles/7042661-moving-from-completions-to-chat-completions-in-the-openai-api).

We'll explore these roles in more depth as they come up - but for now we're going to just stick with the basic role `user`. The `user` role is, as it would seem, the user!

Thirdly, it expects us to specify a model!

We'll use the `gpt-5-mini` model as stated above.

Let's look at an example!



In [151]:
response = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[{"role": "user", "content": "Hello!"}]
)

Let's look at the response object.

In [152]:
response

ChatCompletion(id='chatcmpl-CW0aOq06K8Y5uOms3gibr0v00A6uN', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I help you today? (I can answer questions, draft or edit text, help with coding, explain concepts, plan things, or anything else you need.)', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1761744784, model='gpt-5-mini-2025-08-07', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=109, prompt_tokens=8, total_tokens=117, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=64, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [153]:
print(response.choices[0].message.content)

Hello! How can I help you today? (I can answer questions, draft or edit text, help with coding, explain concepts, plan things, or anything else you need.)


>NOTE: We'll spend more time exploring these outputs later on, but for now - just know that we have access to a tonne of powerful information!

### System Role

Now we can extend our prompts to include a system prompt.

The basic idea behind a system prompt is that it can be used to encourage the behaviour of the LLM, without being something that is directly responded to - let's see it in action!

In the newest OpenAI API, the **system message** still defines the model’s behavior.  
Sometimes it is referred to as an *instruction block*.

Example system prompt for our economics case:

In [154]:
system_prompt = """
You are an experienced economic analyst explaining how inflation and interest rates interact.   
Use 2025 U.S. market context when relevant.
Your answer should not exceed 5 sentences. 
"""
print(system_prompt)

user_prompt = "What is the relationship between inflation and interest rates?"
print(user_prompt)

list_of_prompts = [

    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

irate_response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=list_of_prompts
)

print(irate_response.choices[0].message.content)


You are an experienced economic analyst explaining how inflation and interest rates interact.   
Use 2025 U.S. market context when relevant.
Your answer should not exceed 5 sentences. 

What is the relationship between inflation and interest rates?
Inflation and interest rates are closely related in economic management, primarily through central bank policies such as those of the Federal Reserve. Typically, when inflation rates rise, central banks may increase interest rates to cool down the economy by making borrowing more expensive, thereby reducing spending and slowing inflation. Conversely, if inflation is low, central banks might lower interest rates to encourage borrowing and spending, which can help stimulate the economy. For instance, if by 2025, the U.S. experiences higher inflation rates, the Federal Reserve might respond by hiking interest rates to prevent the economy from overheating. This interaction plays a crucial role in maintaining economic stability by balancing grow

As you can see - the response we get back is very much in line with the system prompt!

Let's try the same user prompt, but with a different system to prompt to see the difference.

In [155]:
system_prompt = """
You are a cool and fun elementary teacher explaining to 6-year olds how inflation and interest rates interact.   
Use 2025 U.S. market context when relevant.
Your answer should not exceed 5 sentences.
"""
print(system_prompt)

user_prompt = "What is the relationship between inflation and interest rates?"
print(user_prompt)

list_of_prompts = [

    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

irate_response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=list_of_prompts
)

print(irate_response.choices[0].message.content)


You are a cool and fun elementary teacher explaining to 6-year olds how inflation and interest rates interact.   
Use 2025 U.S. market context when relevant.
Your answer should not exceed 5 sentences.

What is the relationship between inflation and interest rates?
Imagine if you have a piggy bank where you keep saving money to buy your favorite toy. Now, let's pretend the prices of toys start going up – that’s like inflation, where things cost more. The people who decide on interest rates, kind of like the rules for our piggy banks, can make a change. If they make the interest rate higher, it's like encouraging everyone to save more instead of spending, and that can help stop prices from going up too fast. So, in simple words, when things start to cost more, raising interest rates can help keep everything balanced!


With a simple modification of the system prompt - you can see that we got completely different behaviour, and that's the main goal of prompt engineering as a whole.

Also, congrats, you just engineered your first prompt!

### Few-shot Prompting

Now that we have a basic handle on the `system` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

In [156]:
# Zero-shot prompt
prompt_zero = "Explain how inflation affects interest rate decisions."
list_of_prompts = [
    {"role": "user", "content": prompt_zero}
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=list_of_prompts
)

print('zero-shot response:', response.choices[0].message.content)

zero-shot response: Inflation significantly influences interest rate decisions made by central banks and financial institutions. Here’s how this relationship typically works:

1. **Central Bank Mandates**: Central banks, such as the Federal Reserve in the U.S., often have a dual mandate: to promote maximum employment and stable prices (low inflation). When inflation rises above targeted levels, central banks may take action to curb it.

2. **Interest Rate Adjustments**: To combat rising inflation, central banks may decide to increase interest rates. Higher interest rates make borrowing more expensive, which tends to slow down consumer spending and business investment. This reduction in demand can help lower inflation.

3. **Expectations of Future Inflation**: If consumers and businesses expect inflation to rise, they may adjust their behavior accordingly (e.g., demanding higher wages or preemptively raising prices). Central banks may raise interest rates preemptively to manage these ex

In [ ]:
# Few-shot prompt template

question = "Explain how inflation affects interest rate decisions."

few_shot_prompt = f"""
Example 1:
Q: The price of pizza slices jumps from $2 to $4. What might the central bank do?
A: They turn down the oven heat 🍕🔥 — raise interest rates so people buy fewer slices and cool off the price party.

Example 2:
Q: Interest rates drop and borrowing gets cheaper. What happens at Snack City?
A: Everyone's grabbing extra fries and milkshakes 🍟🥤— cheap credit means more spending, which can make prices rise again.

Now answer:
Q: {question}
"""

list_of_prompts = [
    {"role": "user", "content": few_shot_prompt}
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=list_of_prompts
)

print('few-shot response:', response.choices[0].message.content)

few-shot response: A: When inflation starts to sizzle like a hot griddle 🥓🔥, central banks may hike interest rates to cool things down. Higher rates make borrowing costlier, leading everyone to think twice before splurging on those extra treats. This helps keep prices in check and maintains economic stability!


### Helper functions

We're going to create some helper functions to aid in using the OpenAI API - just to make our lives a bit easier.

> NOTE: Take some time to understand these functions between class!

In [ ]:
from IPython.display import display, Markdown

def get_response(client: OpenAI, messages: list, model: str = "gpt-4o-mini") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

Different way we can do prompting -> using the helper's functions

In [159]:
# Now, show the economic example with both user and assistant prompts
few_shot_prompts = [
    user_prompt("Inflation rises fast. How does the central bank react — dating analogy please!"),
    assistant_prompt("They play hard to get — raise rates — to cool off the economy's over-eager spending habits."),

    user_prompt("What happens when interest rates are too low for too long?"),
    assistant_prompt("Everyone gets too comfortable — too many relationships (loans) form, and eventually hearts (bubbles) break."),

    user_prompt("Explain deflation using a dating metaphor."),
    assistant_prompt("No one's asking anyone out — everyone waits for a better deal, so the economy gets lonely and quiet."),
    # 👇 Here's the actual question we want the model to answer
    user_prompt("Describe quantitative easing")
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=few_shot_prompts
)

print(response.choices[0].message.content)


It's like a dating coach giving you confidence boosters — flooding the market with money to encourage spending and invest in relationships (assets) to revive the economy and spark romance (growth).


### 🏗️ Activity #1:
Mission:
Experiment with how different prompt structures, system, user, and assistant, plus zero-shot and few-shot prompting, can transform an AI’s response.
Your goal: craft the most effective prompt and see how GPT-4-Turbo reacts!

You’ll test how GPT-4-Turbo behaves under four different setups:
1. System/User roles only (Zero-shot)
2. System/User roles + examples (Few-shot)
3. No system role at all (User only)
4. Creative system prompt twist



### Chain of Thought Prompting

We'll head one level deeper and explore the world of Chain of Thought prompting (CoT).

This is a process by which we can encourage the LLM to handle slightly more complex tasks.

Let's look at a simple reasoning based example without CoT.

In [160]:
reasoning_problem = """
The central bank increases the policy rate by 1.5 pp in response to 5 % inflation while nominal wage growth is 3 %.
What happens to real wages?
"""

list_of_prompts = [
    user_prompt(reasoning_problem)
]

reasoning_response = get_response(client, list_of_prompts)
pretty_print(reasoning_response)

Real wages would decrease in this scenario. With inflation at 5%, prices are rising faster than nominal wages, which are only growing at 3%. This means that purchasing power is being eroded as prices outpace wage growth. As a result, real wages, which take inflation into account, would decrease.

Let's see if we can leverage a simple CoT prompt to improve our model's performance on this task:

In [161]:
list_of_prompts = [
    user_prompt(reasoning_problem + "Think step-by-step about how nominal wages, prices, and interest rates interact through the labor market and aggregate demand. Then explain the real wage effect.")
]

reasoning_response = get_response(client, list_of_prompts)
pretty_print(reasoning_response)

When the central bank increases the policy rate by 1.5 pp in response to 5% inflation, it is trying to combat the rising prices by tightening monetary policy. This increase in the policy rate will lead to higher borrowing costs for businesses and individuals, which can slow down economic activity and reduce aggregate demand.

As a result, businesses may not be able to afford to pay higher wages due to the higher borrowing costs and lower demand for their products/services. This can lead to slower growth in nominal wages, despite the fact that inflation is rising.

In this scenario, nominal wage growth is 3%, while inflation is 5%. This means that real wages are actually decreasing, as the purchasing power of wages is being eroded by inflation. Real wages are calculated by adjusting nominal wages for changes in the price level.

So, in this case, the real wage effect would be negative - despite workers receiving a 3% increase in wages, their purchasing power has decreased by more than that due to the 5% inflation rate. This highlights the importance of considering both nominal wages and inflation when assessing changes in real wages.


## 3. Running Comparative Experiment

We'll test combinations of model type (reasoning vs non-reasoning) and prompting style (zero-shot vs few-shot).


In [162]:
# --------------------------------------------------
# 🧩 Comparing GPT Models: Reasoning vs Non-Reasoning
# --------------------------------------------------

from openai import OpenAI
client = OpenAI()

system_prompt = """
You are an experienced economic analyst.
"""

question = """What is the impact of inflation on real wages? Respond in a concise manner."""

prompt_few = f"""
Use this exact format to answer the question:
Example 1:
{{
  "possible_explanation": "Wage catch-up effect",
  "mechanism": "Workers negotiate higher nominal wages to preserve purchasing power as prices rise.",
  "impact_on_wages": "Nominal wages increase roughly in line with inflation, keeping real wages stable in the short run.",
  "time_frame": "Short to medium run",
  "economic_context": "Inflationary periods with strong labor bargaining power or cost-of-living adjustments."
}}

Example 2:
{{
  "possible_explanation": "Real wage erosion",
  "mechanism": "When nominal wages lag behind price growth, workers lose purchasing power.",
  "impact_on_wages": "Real wages decline despite nominal wage increases, reducing workers’ living standards.",
  "time_frame": "Immediate term",
  "economic_context": "High inflation environments with weak wage indexation or rigid labor contracts."
}}

Now answer:
Q: {question}
"""


# --------------------------------------------------
# MODEL 1: GPT-4-turbo  → Non-Reasoning
# --------------------------------------------------
print("\n==============================")
print("MODEL 1: GPT-4-turbo (Non-Reasoning)")
print("==============================\n")

# Zero-shot
answer_nonreasoning_zero_shot = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ],
)
print("Zero-Shot Prompting (no examples):\n")
print("A:", answer_nonreasoning_zero_shot.choices[0].message.content, "\n")



MODEL 1: GPT-4-turbo (Non-Reasoning)

Zero-Shot Prompting (no examples):

A: Inflation erodes the purchasing power of money, meaning that if nominal wages (the face value of wages paid) do not increase at the same rate as inflation, real wages (the purchasing power of those wages) will decline. Essentially, if inflation rises faster than nominal wages, workers can buy less with their earnings, resulting in a decrease in real income and living standards. This can impact consumer spending, savings rates, and overall economic growth. Conversely, if nominal wages increase faster than inflation, real wages rise, enhancing purchasing power and potentially boosting economic activity. 



In [163]:
# --------------------------------------------------
# MODEL 2: GPT-5  → Reasoning
# --------------------------------------------------
print("\n==============================")
print("MODEL 2: GPT-5 (Reasoning-Tuned)")
print("==============================\n")

# Zero-shot
answer_reasoning_zero_shot = client.chat.completions.create(
    model="gpt-5",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ],
)
print("Zero-Shot Prompting (no examples):\n")
print("A:", answer_reasoning_zero_shot.choices[0].message.content, "\n")



MODEL 2: GPT-5 (Reasoning-Tuned)

Zero-Shot Prompting (no examples):

A: - Real wages are nominal wages adjusted for prices; roughly, real wage growth ≈ nominal wage growth − inflation.
- If inflation exceeds nominal wage growth, purchasing power falls; if nominal wages grow faster, it rises.
- Because wages adjust with lags and many aren’t fully indexed, unexpected inflation typically cuts real wages in the short run; COLAs/strong bargaining mitigate, while fixed-pay/minimum-wage workers lose more.
- Non-indexed tax brackets (bracket creep) can further reduce after‑tax real wages; over time, productivity growth is the key driver of sustained real wage gains. 



In [164]:
print("\n==============================")
print("MODEL 1: GPT-4-turbo (Non-Reasoning)")
print("==============================\n")

# Few-shot
answer_nonreasoning_few_shot = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt_few}
    ],
)
print("Few-Shot Prompting (with examples):\n")
print("A:", answer_nonreasoning_few_shot.choices[0].message.content, "\n")


MODEL 1: GPT-4-turbo (Non-Reasoning)

Few-Shot Prompting (with examples):

A: {
  "possible_explanation": "Inflation and real wage dynamics",
  "mechanism": "Inflation reduces the purchasing power of nominal wages unless they are adjusted at the same rate or faster.",
  "impact_on_wages": "Real wages decline if wage increases do not keep pace with inflation, eroding worker purchasing power.",
  "time_frame": "Can be immediate or gradual, depending on inflation rate and wage adjustment frequency.",
  "economic_context": "Periods of sustained inflation without corresponding and timely wage adjustments."
} 



In [165]:
print("\n==============================")
print("MODEL 2: GPT-5 (Reasoning-Tuned)")
print("==============================\n")

# Few-shot
answer_reasoning_few_shot = client.chat.completions.create(
    model="gpt-5",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt_few}
    ],
)
print("Few-Shot Prompting (with examples):\n")
print("A:", answer_reasoning_few_shot.choices[0].message.content, "\n")


MODEL 2: GPT-5 (Reasoning-Tuned)

Few-Shot Prompting (with examples):

A: {
  "possible_explanation": "Real wage erosion",
  "mechanism": "If nominal wage growth trails inflation, purchasing power falls.",
  "impact_on_wages": "Real wages decline until nominal wages catch up.",
  "time_frame": "Immediate to short run",
  "economic_context": "High or accelerating inflation with limited wage indexation or weak bargaining power."
} 




## 4. Evaluation Framework

LLM as a judge


In [166]:
import json

# --------------------------------------------------
# ⚖️ LLM-as-a-Judge Evaluation Script
# --------------------------------------------------

# Define evaluation scale (0–4)
# 0 = completely incorrect / irrelevant
# 1 = partially correct but weak or inaccurate reasoning
# 2 = fair factual accuracy, minimal reasoning
# 3 = accurate and somewhat reasoned
# 4 = highly accurate, clear causal explanation, correct logic

evaluation_prompt = f"""
You are an impartial economics teacher grading two student answers to the same question.

Question:
{question}

Answer A (non-reasoning model):
{answer_nonreasoning_few_shot.choices[0].message.content}

Answer B (reasoning model):
{answer_reasoning_few_shot.choices[0].message.content}

Evaluate both answers on accuracy and reasoning quality on a 0–4 scale:
- 0 = completely incorrect or irrelevant
- 1 = partially correct, but flawed
- 2 = fair factual accuracy, limited reasoning
- 3 = mostly correct, some reasoning
- 4 = fully accurate and clearly reasoned, ability to see the interdependencies between variables.

Return your evaluation as a JSON object in this exact format:
{{
  "Answer A Score": <0-4>,
  "Answer B Score": <0-4>,
  "Better Answer": "A" or "B",
  "Explanation": "Why the better answer is more accurate or reasoned"
}}
"""

# Choose a strong evaluator model (GPT-4.1 is good for judging)
evaluation = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[
        {"role": "system", "content": "You are an impartial LLM evaluator for economics-related answers."},
        {"role": "user", "content": evaluation_prompt}
    ],
)

# Parse and display the evaluation
response_text = evaluation.choices[0].message.content

# Optional: try to parse JSON for structured output
try:
    result = json.loads(response_text)
    print("\nParsed JSON Result:")
    print(json.dumps(result, indent=2))
except json.JSONDecodeError:
    print("\nNote: Could not parse JSON, model may have returned free text instead.")



Parsed JSON Result:
{
  "Answer A Score": 4,
  "Answer B Score": 3,
  "Better Answer": "A",
  "Explanation": "Both answers are factually correct: inflation reduces real wages if nominal wages do not rise as fast. Answer A is slightly better reasoned and more complete \u2014 it notes the role of timely wage adjustments and allows for immediate or gradual effects depending on inflation rate and adjustment frequency, capturing the key interdependency between inflation and wage-setting. Answer B is accurate but narrower (emphasizing the short run) and omits the explicit role of adjustment timing that A highlights, so it receives a lower score."
}


### 🏗️ Activity #2:

Evaluate different prompting strategies using your own example.

## Saving results

In [167]:
# Create markdown content
markdown_content = f"""
# 🧠 Reasoning Model Answer
### Question:
How does inflation affect interest rates and the broader market?

### Model Used:
`gpt-4.1` (Reasoning-tuned)

### Response:
{answer_reasoning_few_shot.choices[0].message.content}

---

*This answer was generated by a reasoning model to illustrate step-by-step economic reasoning.*
"""

output_path='./results.md'
# Save to file
with open(output_path, "w", encoding="utf-8") as f:
    f.write(markdown_content)

print(f"✅ Reasoning model answer saved to: {os.path.abspath(output_path)}")

✅ Reasoning model answer saved to: /Users/katerinag/Documents/AIE/on-ramp/Lesson1/results.md


## Conclusion

- **Few-shot prompts** improve structure and reasoning consistency.  
- **Reasoning models** (like GPT-5-reasoning) deliver more coherent causal explanations between inflation, interest rates, and growth indicators.  
- **Non-reasoning models** (e.g., GPT-5-mini) provide faster, surface-level insights ideal for retrieval or summarization tasks.  
- Future work could add **RAG pipelines** with real-time macroeconomic data or integrate with financial dashboards for live LLM reasoning visualization.